<div style="background: navy; color: white; padding: 20px">

# $\S 4.5$: The rank 2 MMD as an improvement over the rank 1 MMD

## Z Issa Apr 2023

In this notebook, we show how using the rank-2 MMD $\mathcal{D}^2_{\text{sig}}$ as a metric for market regime detection problems can lead to improved performance than by simply using the $\mathcal{D}^1_{\text{sig}}$, as per Subsection 4.5 of the paper "Non-parametric online market regime detection and regime clustering for multidimensional and path-dependent data structures".

<div style="background: navy; color: white; padding: 20px">


### Imports

In [ ]:
from src.generators.config import ModelConfig
from src.testing import TestConfig
from src.testing.discriminators import GeneralMMDDetector
from src.testing.discriminators.config import ProcessorConfig
from src.testing.experiment_functions.mmd_test_functions import get_beliefs_from_config
from src.utils.Config import Config
from src.utils.auxiliary_classes.PathTransformer import PathTransformer, PathTransformerConfig
from src.utils.auxiliary_classes.RegimePartitioner import RegimePartitioner, RegimePartitionerConfig
from src.testing.experiment_functions.mmd_test_functions import alpha_score_function
from src.utils.helper_functions.plot_helper_functions import plot_regime_change_path
from src.testing.experiment_functions.plot_result_functions import plot_path_experiment_result, plot_path_test_threshold
from src.utils.helper_functions.test_helper_functions import get_memberships, get_alphas, get_sub_paths, \
    get_grouped_paths

<div style="background: navy; color: white; padding: 20px">

    
## 1. Set configurations

For a deeper explanation for what each of the configuration variables does, see the notebook <code>5-higher-rank-mmd.ipynb</code>.

In [ ]:
test_config             = TestConfig()
path_transformer_config = PathTransformerConfig()
regime_config           = RegimePartitionerConfig()

year_mesh = 252          # Grid size parameter
time      = 1/12         # Fraction of a year to simulate over (belief paths)
T         = 20           # Length of regime path
dim       = 5            # Dimensionality of path objects
dt        = 1/year_mesh
S0        = [1. for _ in range(dim)]

model_config = ModelConfig()
model_config.override_args({
    "year_mesh": year_mesh,
    "attach_volatility": False
})

path_transformer_config.set_transformations({
    "standardise_path_transform":    (True , 0, {"s_type": "initial"}),
    "time_normalisation_transform":  (True , 0, {}),
    "time_difference_transform":     (False, 0, {}),
    "difference_transform":          (False, 0, {}),
    "translation_transform":         (False , 0, {}),
    "scaling_transform":             (False, 0, {"sigmas": [(1/dt)**0.5 for _ in range(dim)]}),
    "cumulant_transform":            (False , 2, {}),
    "increment_transform":           (False, 2, {}),
    "lead_lag_transform":            (False, 3, {}),
    "invisibility_transform":        (False, 4, {}),
})

path_transformer_config.compute_pathwise_signature_transform = False
path_transformer_config.signature_order = 8

test_config.override_args({
    "n_steps"          : int(time*year_mesh),
    "n_paths"          : 10,
    "offset"           : 0,
    "weight_factor"    : 1,
    "belief_models"    : ["gbm"],
    "model_pair_names" : ["gbm", "rBergomi"],
    "belief_params"    : [[[0., 0.2] for _ in range(dim)]],
    "model_pair_params": [[[0., 0.2] for _ in range(dim)], [[0.1, 0.1, -0.7, 0.3] for _ in range(dim)]],
    "path_bank_size"   : 100000
})

mmd1_processor_config = ProcessorConfig()
mmd2_processor_config = ProcessorConfig()

sigma0 = 0.5
sigma1 = 1

mmd1_processor_config.override_args({
    "generalmmddetector_kwargs": Config(**{
        "normalise": False,
        "n_tests": 512,
        "n_evaluations": 1,
        "metric_kwargs": Config(**{
            "kernel_type": "rbf",
            "metric_type": "mmd",
            "sigmas": [sigma0],  
            "dyadic_orders": [0],  
            "lambd": 1
        }),
        "evaluator_kwargs": Config(**{
            "pct_ignore": 0.1
        })
    })
})

mmd2_processor_config.override_args({
    "generalmmddetector_kwargs": Config(**{
        "normalise": False,
        "n_tests": 512,
        "n_evaluations": 1,
        "metric_kwargs": Config(**{
            "kernel_type": "rbf",
            "metric_type": "mmd",
            "sigmas": [sigma0, sigma1],  
            "dyadic_orders": [0, 0],  
            "lambd": 1
        }),
        "evaluator_kwargs": Config(**{
            "pct_ignore": 0.1
        })
    })
})

regime_config.override_args({
    "n_regime_changes": 5,
    "f_length_scale"  : 0.5,
    "type"            : "random_on_off_steps",
    "r_on_args"       : ["poisson", 0.25],
    "r_off_args"      : ["poisson", 5],
    "r_min_distance"  : 40
})
        
beliefs, belief_details, model_pairs = get_beliefs_from_config(test_config, model_config)
path_transformer = PathTransformer(path_transformer_config)

<div style="background: navy; color: white; padding: 20px">

## 2. Initialize detector objects

In [ ]:
mmd1_detector = GeneralMMDDetector(
    beliefs          = beliefs, 
    path_details     = belief_details,
    path_transformer = path_transformer,
    processor_config = mmd1_processor_config,
    test_config      = test_config
)

mmd2_detector = GeneralMMDDetector(
    beliefs          = beliefs, 
    path_details     = belief_details,
    path_transformer = path_transformer,
    processor_config = mmd2_processor_config,
    test_config      = test_config
)

<div style="background: navy; color: white; padding: 20px">

## 3. Build regime-changed path

In [ ]:
# Generate path for testing
regime_partitioner = RegimePartitioner(model_config.year_mesh, regime_config)
regime_partitioner.generate_regime_partitions(T=T, n_steps=test_config.n_steps)
test_path = regime_partitioner.generate_regime_change_path(model_pairs, S0)

In [ ]:
_, regime_changes, _ = regime_partitioner.changes_to_times()
path_splits_time = [item for sublist in [[r[0], r[1] + 1/year_mesh] for r in regime_changes] for item in sublist]
path_splits_mesh = [int(p*year_mesh) for p in path_splits_time]

plot_regime_change_path(test_path, regime_changes, log_returns=False, one_dim=True)

In [ ]:
plot_regime_change_path(test_path, regime_changes, log_returns=True, one_dim=True)

<div style="background: navy; color: white; padding: 20px">

## 4. Calculate scores with respect to each detector

In [ ]:
mmd1_scores = mmd1_detector.evaluate_path(test_path, evaluation="total")
mmd2_scores = mmd2_detector.evaluate_path(test_path, evaluation="total")

<div style="background: navy; color: white; padding: 20px">

## 5. Get scores

In [ ]:
# Report scores
n_steps        = test_config.n_steps
n_paths        = test_config.n_paths
sub_paths      = get_sub_paths(test_path, n_steps, 0)
mmd_paths      = get_grouped_paths(sub_paths, n_paths)

memberships    = get_memberships(mmd_paths)
c_alpha_1      = mmd1_detector.critical_value
c_alpha_2= mmd2_detector.critical_value
mmd_alphas_1   = get_alphas(memberships, mmd1_scores, c_alpha_1)
mmd_alphas_2   = get_alphas(memberships, mmd2_scores, c_alpha_2)


mmd1_scores_dict = alpha_score_function(
    regime_changes = regime_partitioner.regime_changes,
    path_length    = len(test_path),
    memberships    = memberships,
    test_alphas    = mmd_alphas_1[0],
    test_data      = [n_steps, 0, n_paths]
)

print("mmd1_Detector scores")
for k, v in mmd1_scores_dict.items():
    print("{}: {:.4f}%".format(k, v*100))

mmd2_scores_dict = alpha_score_function(
    regime_changes = regime_partitioner.regime_changes,
    path_length    = len(test_path),
    memberships    = memberships,
    test_alphas    = mmd_alphas_2[0],
    test_data      = [n_steps, 0, n_paths]
)

print("\nmmd2_Detector scores")
for k, v in mmd2_scores_dict.items():
    print("{}: {:.4f}%".format(k, v*100))

<div style="background: navy; color: white; padding: 20px">


## 6. Plot results

In [ ]:
plot_path_experiment_result(test_path, mmd1_scores, path_splits_mesh, diff=False, one_dim=True)
plot_path_experiment_result(test_path, mmd2_scores, path_splits_mesh, diff=False, one_dim=True)

In [ ]:
plot_path_test_threshold(sub_paths, mmd_alphas_1[0], path_splits_time, one_dim=True)
plot_path_test_threshold(sub_paths, mmd_alphas_2[0], path_splits_time, one_dim=True)